# INTRODUCTION
This notebook combines and explores daily flight data from Dallas Fort Worth (DFW) airport and daily weather data for DFW from Jan 1 2018 to Nov 17, 2023. The purpose is to produce models that can predict daily flight traffic, such as the number of total, ontime, cancelled, and delayed flights at Dallas Fort Worth Airport (DFW) from historical flight and weather data. This analysis is exploratory, but could be valuable to DFW airport management.

The daily weather data were obtained from a friend, Monte Lunacek, at the National Renewable Energy Laboratory (NREL) who obtained the data from the National Oceanic and Atmospheric Administration (NOAA), High-Resolution Rapid Refresh (HRRR) Data Archive: AWS OPen Data Program. The weather flight data was scaped from the Bureau of Transporation Statistics using Selenium.. 

In addition to merging the daily flight and weather data, this notebook engineers many features potentially relevant to flight traffic, such as time, date, and holiday features, and lag columns for total flights and cancelled flights. Column lists are created for convenient handling of features and targets. The final dataset is saved as "datav3/daily/daily_flights_and_weather_merged.parquet" for use in machine learning peformed in other notebooks.



## Libraries

In [168]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import holidays
import datetime as dt

import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, FunctionTransformer
from sklearn.decomposition import PCA

DAILY_DATA_PATH = "data.v3/daily" 

pd.set_option("display.width", 600)

# DAILY FLIGHT DATA

In [169]:
flights_d = pd.read_parquet(os.path.join(DAILY_DATA_PATH, "daily_flights.parquet"))
print(f"Daily flights columns: {flights_d.columns.tolist()}")
print('Daily flights import shape:', flights_d.shape)

Daily flights columns: ['flights_cancel', 'flights_delay', 'flights_ontime', 'flights_arr', 'flights_dep', 'flights_arr_A', 'flights_arr_B', 'flights_arr_C', 'flights_arr_D', 'flights_arr_E', 'flights_dep_A', 'flights_dep_B', 'flights_dep_C', 'flights_dep_D', 'flights_dep_E', 'flights_arr_cancel', 'flights_arr_delay', 'flights_arr_ontime', 'flights_dep_cancel', 'flights_dep_delay', 'flights_dep_ontime']
Daily flights import shape: (2147, 21)


## Explore daily flights data

In [170]:
flights_d.describe().round().T

,count,mean,std,min,25%,50%,75%,max
flights_cancel,2147.0,40.0,108.0,0.0,4.0,10.0,26.0,1330.0
flights_delay,2147.0,315.0,191.0,0.0,184.0,272.0,404.0,1270.0
flights_ontime,2147.0,1326.0,276.0,33.0,1228.0,1392.0,1508.0,1848.0
flights_arr,2147.0,840.0,138.0,17.0,795.0,867.0,932.0,1106.0
flights_dep,2147.0,841.0,138.0,16.0,792.0,869.0,934.0,1080.0
flights_arr_A,2147.0,154.0,27.0,0.0,143.0,159.0,173.0,227.0
flights_arr_B,2147.0,225.0,38.0,0.0,203.0,228.0,255.0,311.0
flights_arr_C,2147.0,172.0,40.0,0.0,149.0,182.0,202.0,244.0
flights_arr_D,2147.0,108.0,24.0,16.0,93.0,106.0,127.0,169.0
flights_arr_E,2147.0,181.0,48.0,1.0,142.0,185.0,221.0,280.0


In [171]:
flights_d.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2147 entries, 2018-01-01 to 2023-11-17
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   flights_cancel      2147 non-null   float64
 1   flights_delay       2147 non-null   float64
 2   flights_ontime      2147 non-null   float64
 3   flights_arr         2147 non-null   float64
 4   flights_dep         2147 non-null   float64
 5   flights_arr_A       2147 non-null   float64
 6   flights_arr_B       2147 non-null   float64
 7   flights_arr_C       2147 non-null   float64
 8   flights_arr_D       2147 non-null   float64
 9   flights_arr_E       2147 non-null   float64
 10  flights_dep_A       2147 non-null   float64
 11  flights_dep_B       2147 non-null   float64
 12  flights_dep_C       2147 non-null   float64
 13  flights_dep_D       2147 non-null   float64
 14  flights_dep_E       2147 non-null   float64
 15  flights_arr_cancel  2147 non-null   f

In [172]:
flights_d.head().T

timestamp,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05
flights_cancel,115.0,76.0,38.0,82.0,44.0
flights_delay,602.0,649.0,338.0,215.0,231.0
flights_ontime,985.0,1076.0,1349.0,1409.0,1439.0
flights_arr,870.0,884.0,863.0,851.0,857.0
flights_dep,832.0,917.0,862.0,855.0,857.0
flights_arr_A,177.0,178.0,177.0,165.0,172.0
flights_arr_B,298.0,279.0,276.0,282.0,283.0
flights_arr_C,187.0,190.0,193.0,192.0,192.0
flights_arr_D,100.0,110.0,94.0,91.0,93.0
flights_arr_E,108.0,127.0,123.0,121.0,117.0


In [173]:
flights_d.tail().T

timestamp,2023-11-13,2023-11-14,2023-11-15,2023-11-16,2023-11-17
flights_cancel,6.0,1.0,6.0,0.0,0.0
flights_delay,214.0,53.0,0.0,0.0,0.0
flights_ontime,1672.0,1767.0,1623.0,65.0,33.0
flights_arr,945.0,910.0,810.0,41.0,17.0
flights_dep,947.0,911.0,819.0,24.0,16.0
flights_arr_A,161.0,161.0,161.0,7.0,0.0
flights_arr_B,249.0,237.0,225.0,0.0,0.0
flights_arr_C,210.0,210.0,191.0,2.0,0.0
flights_arr_D,134.0,125.0,141.0,25.0,16.0
flights_arr_E,191.0,177.0,92.0,7.0,1.0


### Missing values in daily flight data

In [174]:
flights_d['date'] = flights_d.index
flights_d['year'] = flights_d['date'].dt.year
missing = flights_d.groupby('year').apply(lambda x: x.isnull().sum())
print("Missing values per year:", missing)

Missing values per year:       flights_cancel  flights_delay  flights_ontime  flights_arr  flights_dep  flights_arr_A  flights_arr_B  flights_arr_C  flights_arr_D  flights_arr_E  ...  flights_dep_D  flights_dep_E  flights_arr_cancel  flights_arr_delay  flights_arr_ontime  flights_dep_cancel  flights_dep_delay  flights_dep_ontime  date  year
year                                                                                                                                                      ...                                                                                                                                                                
2018               0              0               0            0            0              0              0              0              0              0  ...              0              0                   0                  0                   0                   0                  0                   0     0     0
2019               0 

/tmp/ipykernel_234801/938010236.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing = flights_d.groupby('year').apply(lambda x: x.isnull().sum())


# DAILY WEATHER
Source: NOAA Global Systems Laboratory, High-Resolution Rapid Refresh (HRRR) Data Archive: AWS OPen Data Program\
Data dictionary: [HRRR Zarr Variable List](https://mesowest.utah.edu/html/hrrr/zarr_documentation/html/zarr_variables.html)

In [175]:
weather_d = pd.read_parquet(os.path.join(DAILY_DATA_PATH, "daily_weather.parquet"))
print(f"Weather columns: {weather_d.columns.tolist()}")
print("Daily weather import shape:", weather_d.shape)

Weather columns: ['wx_temperature_max', 'wx_temperature_min', 'wx_apcp', 'wx_prate', 'wx_asnow', 'wx_frozr', 'wx_vis', 'wx_gust', 'wx_maxref', 'wx_cape', 'wx_lftx', 'wx_wind_speed', 'wx_wind_direction']
Daily weather import shape: (4809, 13)


## Weather Info and Descriptives

In [176]:
print("Weather null count and data types:")
print(weather_d.info())

print("\n\nWeather descriptives", weather_d.describe().round().T)

Weather null count and data types:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4809 entries, 2010-01-01 to NaT
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   wx_temperature_max  1692 non-null   float64
 1   wx_temperature_min  1692 non-null   float64
 2   wx_apcp             1692 non-null   float64
 3   wx_prate            1692 non-null   float64
 4   wx_asnow            1692 non-null   float64
 5   wx_frozr            1692 non-null   float64
 6   wx_vis              1692 non-null   float64
 7   wx_gust             1692 non-null   float64
 8   wx_maxref           1692 non-null   float32
 9   wx_cape             1692 non-null   float32
 10  wx_lftx             1692 non-null   float32
 11  wx_wind_speed       1692 non-null   float64
 12  wx_wind_direction   1692 non-null   float32
dtypes: float32(4), float64(9)
memory usage: 450.8 KB
None


Weather descriptives                      count   m

## Weather Head and Tail

In [177]:
print("Head\n", weather_d.head().T)
print("\n\nTail\n", weather_d.tail().T)

Head
                     2010-01-01  2010-01-02  2010-01-03  2010-01-04  2010-01-05
wx_temperature_max         NaN         NaN         NaN         NaN         NaN
wx_temperature_min         NaN         NaN         NaN         NaN         NaN
wx_apcp                    NaN         NaN         NaN         NaN         NaN
wx_prate                   NaN         NaN         NaN         NaN         NaN
wx_asnow                   NaN         NaN         NaN         NaN         NaN
wx_frozr                   NaN         NaN         NaN         NaN         NaN
wx_vis                     NaN         NaN         NaN         NaN         NaN
wx_gust                    NaN         NaN         NaN         NaN         NaN
wx_maxref                  NaN         NaN         NaN         NaN         NaN
wx_cape                    NaN         NaN         NaN         NaN         NaN
wx_lftx                    NaN         NaN         NaN         NaN         NaN
wx_wind_speed              NaN         NaN    

## Missing values in daily weather

In [178]:
# count the missing values in each column aggregated by year
weather_d['date'] = weather_d.index
weather_d['year'] = weather_d['date'].dt.year
missing = weather_d.groupby('year').apply(lambda x: x.isnull().sum())
print("Missing values by year and variable\n", missing.T)

# Show rows where the year is 2022 and there are missing values in the weather data
print("\nAnomalous missing values on October 1st, 2022\n")
print(weather_d[(weather_d['year'] == 2022) & (weather_d.isnull().any(axis=1))])

Missing values by year and variable
 year                2010.0  2011.0  2012.0  2013.0  2014.0  2015.0  2016.0  2017.0  2018.0  2019.0  2020.0  2021.0  2022.0  2023.0
wx_temperature_max     365     365     366     365     365     365     366     365     193       0       0       0       1       0
wx_temperature_min     365     365     366     365     365     365     366     365     193       0       0       0       1       0
wx_apcp                365     365     366     365     365     365     366     365     193       0       0       0       1       0
wx_prate               365     365     366     365     365     365     366     365     193       0       0       0       1       0
wx_asnow               365     365     366     365     365     365     366     365     193       0       0       0       1       0
wx_frozr               365     365     366     365     365     365     366     365     193       0       0       0       1       0
wx_vis                 365     365     366    

/tmp/ipykernel_234801/1147223249.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing = weather_d.groupby('year').apply(lambda x: x.isnull().sum())


## Drop years and rows with no weather data

In [179]:
weather_d_incomplete = weather_d[weather_d.isnull().any(axis=1)].copy()
weather_d_complete = weather_d[~weather_d.isnull().any(axis=1)].copy()

print("Number of weather rows with missing values:", weather_d_incomplete.shape[0])
print("\nDaily weather final shape:", weather_d_complete.shape)

Number of weather rows with missing values: 3117

Daily weather final shape: (1692, 15)


## Add quadratic weather columns

In [180]:
for col in ['wx_temperature_max', 'wx_temperature_min', 'wx_apcp', 'wx_prate', 'wx_asnow', 'wx_frozr', 'wx_vis', 'wx_gust', 'wx_maxref', 'wx_cape', 'wx_lftx', 'wx_wind_speed', 'wx_wind_direction']:
    weather_d_complete.loc[:, f'{col}_sq'] = weather_d_complete[col] ** 2

## Updated weather data head and tail

In [181]:
print("Head\n", weather_d_complete.head().T)
print("\n\nTail\n", weather_d_complete.tail().T)

Head
                                 2018-07-13           2018-07-14           2018-07-15           2018-07-16           2018-07-17
wx_temperature_max               114.65921            116.12171             119.0466           119.834155           123.884155
wx_temperature_min               86.081519            75.439008            80.635596            81.717804            82.303101
wx_apcp                             0.0104               0.3344               0.0004                0.006                0.006
wx_prate                               0.0             0.113386                  0.0                  0.0                  0.0
wx_asnow                               0.0                  0.0                  0.0                  0.0                  0.0
wx_frozr                               0.0                  0.0                  0.0                  0.0                  0.0
wx_vis                           19.883879             4.846695            13.110933            15.037183

# MERGE FLIGHT AND WEATHER DATA
Note: The weather data covers a shorter time period than the flights data. Can we get more weather data?

In [134]:
# Merge flights and weather data
df = pd.merge(flights_d.drop(['year', 'date'], axis=1), 
              weather_d_complete.drop(['year', 'date'], axis=1), 
              left_index=True, right_index=True, how='inner')

print("flights_d shape:", flights_d.shape)
print("weather_d_complete shape:", weather_d_complete.shape)
print("merged shape:", df.shape)
print('\nHEAD\n', df.head(2).T)
print('\nTAIL\n', df.tail(2).T)


flights_d shape: (2147, 23)
weather_d_complete shape: (1692, 28)
merged shape: (1692, 47)

HEAD
                          2018-07-13    2018-07-14
flights_cancel         4.800000e+01  1.200000e+01
flights_delay          4.810000e+02  3.200000e+02
flights_ontime         1.382000e+03  1.431000e+03
flights_arr            9.490000e+02  8.810000e+02
flights_dep            9.620000e+02  8.820000e+02
flights_arr_A          1.940000e+02  1.810000e+02
flights_arr_B          2.620000e+02  2.540000e+02
flights_arr_C          2.210000e+02  2.100000e+02
flights_arr_D          1.110000e+02  1.070000e+02
flights_arr_E          1.610000e+02  1.290000e+02
flights_dep_A          2.050000e+02  1.870000e+02
flights_dep_B          2.650000e+02  2.550000e+02
flights_dep_C          2.220000e+02  2.070000e+02
flights_dep_D          1.040000e+02  9.800000e+01
flights_dep_E          1.660000e+02  1.350000e+02
flights_arr_cancel     3.800000e+01  9.000000e+00
flights_arr_delay      2.020000e+02  1.350000e+02
fli

# FEATURE ENGINEERING

## Add time, date, and holiday features

In [151]:
import datetime as dt
from dateutil.easter import easter

# create a date column from index
df['date'] = df.index
df['year'] = df['date'].dt.year
# df['year_c'] = df['date'].dt.year.astype('category')
df['month'] = df['date'].dt.strftime('%B')
df['day_of_week'] = df['date'].dt.strftime('%A')
df['day_of_month'] = df['date'].dt.day

# Create an ordinal date column (days since 1/1/1)
df['ordinal_date'] = df['date'].map(dt.datetime.toordinal)

# add season column
def get_season(date):
    if date.month in [12, 1, 2]:
        return 'winter'
    elif date.month in [3, 4, 5]:
        return 'spring'
    elif date.month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

df['season'] = df['date'].apply(get_season)

# Holidays
years = range(df['year'].min(), df['year'].max()+1)
us_holidays = holidays.US(years = years)

# Reverse dictionaries of US Holidays spanning the years in the dataset
# for year in years:
#     globals()[f"holidays_{year}"] = {v: k for k, v in holidays.US(years=year).items()}

# add holiday column
def check_holiday(date):
    # us_holidays = holidays.US(years = date.year)
    return us_holidays.get(date, "Not a Holiday")
df['holiday'] = df['date'].apply(check_holiday)

# add "halloween" column
def halloween(date):
    if date.month == 10 and date.day == 31:
        return 'yes'
    else:
        return 'no'
df['halloween'] = df['date'].apply(halloween)

# add "jan_2" column
def jan_2(date):
    if date.month == 1 and date.day == 2:
        return 'yes'
    else:
        return 'no'
df['jan_2'] = df['date'].apply(jan_2)
    
# add "jan_3" column
def jan_3(date):
    if date.month == 1 and date.day == 3:
        return 'yes'
    else:
        return 'no'
df['jan_3'] = df['date'].apply(jan_3)
    

# add column for the day before easter
def day_before_easter(date):
    normalized_input_date = date.date()
    day_before_easter_date = easter(date.year) - dt.timedelta(days=1)
    if normalized_input_date == day_before_easter_date:
        return 'yes'
    else:
        return 'no'
df['day_before_easter'] = df['date'].apply(day_before_easter)

# add column for the Xmas Eve
def xmas_eve(date):
    if date.month == 12 and date.day == 24:
        return 'yes'
    else:
        return 'no'
df['xmas_eve'] = df['date'].apply(xmas_eve)

# add column for New Year's eve
def new_years_eve(date):
    if date.month == 12 and date.day == 31:
        return 'yes'
    else:
        return 'no'
df['new_years_eve'] = df['date'].apply(new_years_eve)

# add "days_until_Xmas" column
def days_until_xmas(date):
    xmas = pd.to_datetime(f"{date.year}-12-25")
    if date > xmas:
        xmas = pd.to_datetime(f"{date.year+1}-12-25")
    return (xmas - date).days
df['days_until_xmas'] = df['date'].apply(days_until_xmas)


# add "days_until_thanksgiving" column
def days_until_thanksgiving(date):
    # Thanksgiving is the 4th Thursday of November
    # https://www.timeanddate.com/holidays/us/thanksgiving-day
    thanksgiving = pd.to_datetime(f"{date.year}-11-01")
    while thanksgiving.weekday() != 3:
        thanksgiving += pd.Timedelta(days=1)
    thanksgiving += pd.Timedelta(days=21)
    return (thanksgiving - date).days
df['days_until_thanksgiving'] = df['date'].apply(days_until_thanksgiving)

# add "days_until_July_4th" column
def days_until_july_4th(date):
    july_4th = pd.to_datetime(f"{date.year}-07-04")
    if date > july_4th:
        july_4th = pd.to_datetime(f"{date.year+1}-07-04")
    return (july_4th - date).days
df['days_until_july_4th'] = df['date'].apply(days_until_july_4th)

# add "days_until_labor_day" column
def days_until_labor_day(date):
    # Labor Day is the first Monday of September
    labor_day = pd.to_datetime(f"{date.year}-09-01")
    while labor_day.weekday() != 0:
        labor_day += pd.Timedelta(days=1)
    return (labor_day - date).days
df['days_until_labor_day'] = df['date'].apply(days_until_labor_day)

# add "days_until_memorial_day" column
def days_until_memorial_day(date):
    # Memorial Day is the last Monday of May
    memorial_day = pd.to_datetime(f"{date.year}-05-01")
    if date > memorial_day:
        memorial_day = pd.to_datetime(f"{date.year+1}-05-01")
    return (memorial_day - date).days
df['days_until_memorial_day'] = df['date'].apply(days_until_memorial_day)

# add covid column. The WHO declared COVID-19 a pandemic on March 11, 2020. Record-breaking Thanksgiving travel in 2023 suggests a full recovery by 11/23/23.
mask = (df['date'] >= '2020-03-11') & (df['date'] <= '2023-11-23')
df['covid'] = np.where(mask, 'yes', 'no')

# Move date-related variables to front of dataframe
cols = df.columns.tolist()
date_cols= ['date', 'covid', 'ordinal_date', 'year', 'month', 'day_of_month', 'day_of_week', 'season', 'holiday', 'halloween', 'xmas_eve', 'new_years_eve', 'jan_2', 'jan_3', 'day_before_easter', 'days_until_xmas', 'days_until_thanksgiving', 'days_until_july_4th', 'days_until_labor_day', 'days_until_memorial_day']
col_order = date_cols + [col for col in cols if col not in date_cols]
df = df[col_order]


# Verify the day before easter dates make sense
print("Day before easter dates:", df[df['day_before_easter'] == 'yes']['date'].tolist())

# Preview the dataframe
print("\nDate-related columns:\n")
df[date_cols].head().T

Day before easter dates: [Timestamp('2019-04-20 00:00:00'), Timestamp('2020-04-11 00:00:00'), Timestamp('2021-04-03 00:00:00'), Timestamp('2022-04-16 00:00:00')]

Date-related columns:



,2018-07-20,2018-07-21,2018-07-22,2018-07-23,2018-07-24
date,2018-07-20 00:00:00,2018-07-21 00:00:00,2018-07-22 00:00:00,2018-07-23 00:00:00,2018-07-24 00:00:00
covid,no,no,no,no,no
ordinal_date,736895,736896,736897,736898,736899
year,2018,2018,2018,2018,2018
month,July,July,July,July,July
day_of_month,20,21,22,23,24
day_of_week,Friday,Saturday,Sunday,Monday,Tuesday
season,summer,summer,summer,summer,summer
holiday,Not a Holiday,Not a Holiday,Not a Holiday,Not a Holiday,Not a Holiday
halloween,no,no,no,no,no


## Add flights total and percent delayed, ontime, cancelled columns

In [138]:
# Calculate percentages for total flights
df['flights_total'] = df['flights_cancel'] + df['flights_delay'] + df['flights_ontime']
df['flights_cancel_pct'] = 100*df['flights_cancel'] / df['flights_total']
df['flights_delay_pct'] = 100*df['flights_delay'] / df['flights_total']
df['flights_ontime_pct'] = 100*df['flights_ontime'] / df['flights_total']

# Calculate percentages for arrivals
df['flights_arr_delay_pct'] = 100*df['flights_arr_delay'] / df['flights_arr']
df['flights_arr_ontime_pct'] = 100*df['flights_arr_ontime'] / df['flights_arr']
df['flights_arr_cancel_pct'] = 100*df['flights_arr_cancel'] / df['flights_arr']

# Calculate percentages for departures
df['flights_dep_delay_pct'] = 100*df['flights_dep_delay'] / df['flights_dep']
df['flights_dep_ontime_pct'] = 100*df['flights_dep_ontime'] / df['flights_dep']
df['flights_dep_cancel_pct'] = 100*df['flights_dep_cancel'] / df['flights_dep']

print(df.head().T)

print('shape:', df.shape)

                                 2018-07-13           2018-07-14           2018-07-15           2018-07-16           2018-07-17
date                    2018-07-13 00:00:00  2018-07-14 00:00:00  2018-07-15 00:00:00  2018-07-16 00:00:00  2018-07-17 00:00:00
covid                                    no                   no                   no                   no                   no
ordinal_date                         736888               736889               736890               736891               736892
year                                   2018                 2018                 2018                 2018                 2018
month                                  July                 July                 July                 July                 July
...                                     ...                  ...                  ...                  ...                  ...
flights_arr_ontime_pct            74.710221            83.654938            82.043011            80.8351

## Add 7 lag features each for cancelled, delayed, and ontime flights using 1 day steps
The lag columns will be used exclusively as features. Because total flights can be derived from ontime, delayed, and cancelled flights, we will not create a lag column for total flights.

In [139]:
lag_cols = []

cols_to_lag = ['flights_cancel', 'flights_delay', 'flights_ontime']
for col in cols_to_lag:
    for lag in [1, 2, 3, 4, 5, 6, 7]:
        df[f"{col}_lag_{lag}"] = df[col].shift(lag)
        lag_cols.append(f"{col}_lag_{lag}")

df.dropna(inplace=True)

print(df.shape)
print(f"Lag cols: {lag_cols}")
print("\n", df[['flights_cancel', 'flights_cancel_lag_1', 'flights_cancel_lag_2', 'flights_cancel_lag_3']].head().T)
print("\n", df[['flights_delay', 'flights_delay_lag_1', 'flights_delay_lag_2', 'flights_delay_lag_3']].head().T)
print("\n", df[['flights_ontime', 'flights_ontime_lag_1', 'flights_ontime_lag_2', 'flights_ontime_lag_3']].head().T)

(1685, 98)
Lag cols: ['flights_cancel_lag_1', 'flights_cancel_lag_2', 'flights_cancel_lag_3', 'flights_cancel_lag_4', 'flights_cancel_lag_5', 'flights_cancel_lag_6', 'flights_cancel_lag_7', 'flights_delay_lag_1', 'flights_delay_lag_2', 'flights_delay_lag_3', 'flights_delay_lag_4', 'flights_delay_lag_5', 'flights_delay_lag_6', 'flights_delay_lag_7', 'flights_ontime_lag_1', 'flights_ontime_lag_2', 'flights_ontime_lag_3', 'flights_ontime_lag_4', 'flights_ontime_lag_5', 'flights_ontime_lag_6', 'flights_ontime_lag_7']

                       2018-07-20  2018-07-21  2018-07-22  2018-07-23  2018-07-24
flights_cancel              24.0        30.0        18.0        48.0        14.0
flights_cancel_lag_1        25.0        24.0        30.0        18.0        48.0
flights_cancel_lag_2        16.0        25.0        24.0        30.0        18.0
flights_cancel_lag_3        24.0        16.0        25.0        24.0        30.0

                      2018-07-20  2018-07-21  2018-07-22  2018-07-23  201

## Add a random feature to test RFE

In [140]:
# Add a random variable to df_lag with repeatable results
np.random.seed(42)
df['random'] = np.random.randint(0, 100, df.shape[0])

## Add forecast columns, targets shifted forward by one day

In [141]:
for target in ['flights_total', 'flights_cancel', 'flights_delay', 'flights_ontime']:
    df[f"{target}_next_day"] = df[target].shift(-1) # next day's value

## Column Groups (flights, weather, date)

In [143]:
# Flights column groups
flights_terminal_cols = ['flights_arr_A', 'flights_arr_B', 'flights_arr_C', 'flights_arr_D', 'flights_arr_E',
                         'flights_dep_A', 'flights_dep_B', 'flights_dep_C', 'flights_dep_D', 'flights_dep_E']

flights_non_terminal_cols = ['flights_total', 'flights_cancel', 'flights_delay', 'flights_ontime',
                             'flights_arr_ontime', 'flights_arr_delay', 'flights_arr_cancel',
                             'flights_dep_ontime', 'flights_dep_delay', 'flights_dep_cancel']

flights_percentage_cols = ['flights_cancel_pct', 'flights_delay_pct', 'flights_ontime_pct',
                            'flights_arr_delay_pct', 'flights_arr_ontime_pct', 'flights_arr_cancel_pct',
                            'flights_dep_delay_pct', 'flights_dep_ontime_pct', 'flights_dep_cancel_pct']

flights_forecast_cols = ['flights_total_next_day', 'flights_cancel_next_day', 'flights_delay_next_day', 'flights_ontime_next_day']

# Date column groups
date_cols = ['date', 'covid', 'ordinal_date', 'year', 'month', 'day_of_month', 'day_of_week', 'season', 'holiday', 'halloween', 'xmas_eve', 'new_years_eve', 'jan_2', 'jan_3', 'day_before_easter', 'days_until_xmas', 'days_until_thanksgiving', 'days_until_july_4th', 'days_until_labor_day', 'days_until_memorial_day']

# Weather column groups
weather_cols = ['wx_temperature_max', 'wx_temperature_min', 'wx_apcp', 'wx_prate', 'wx_asnow', 'wx_frozr', 'wx_vis', 'wx_gust', 'wx_maxref', 'wx_cape', 'wx_lftx', 'wx_wind_speed', 'wx_wind_direction']
weather_cols_sq = ['wx_temperature_max_sq', 'wx_temperature_min_sq', 'wx_apcp_sq', 'wx_prate_sq', 'wx_asnow_sq', 'wx_frozr_sq', 'wx_vis_sq', 'wx_gust_sq', 'wx_maxref_sq', 'wx_cape_sq', 'wx_lftx_sq', 'wx_wind_speed_sq', 'wx_wind_direction_sq']

# Lag column groups
lag_cols =  ['flights_cancel_lag_1', 'flights_cancel_lag_2', 'flights_cancel_lag_3', 'flights_cancel_lag_4', 'flights_cancel_lag_5', 'flights_cancel_lag_6', 'flights_cancel_lag_7',
             'flights_delay_lag_1', 'flights_delay_lag_2', 'flights_delay_lag_3', 'flights_delay_lag_4', 'flights_delay_lag_5', 'flights_delay_lag_6', 'flights_delay_lag_7',
             'flights_ontime_lag_1', 'flights_ontime_lag_2', 'flights_ontime_lag_3', 'flights_ontime_lag_4', 'flights_ontime_lag_5', 'flights_ontime_lag_6', 'flights_ontime_lag_7']

## Export data to parquet

In [144]:
# Export the final dataframe
df.to_parquet(os.path.join(DAILY_DATA_PATH, "daily_flights_and_weather_merged.parquet"))